In [11]:
import deepspeech
from datasets import load_dataset
import numpy as np
import pandas as pd




In [ ]:
PATH_TRANSCRIPTIONS = "./../eval/transcriptions.csv"
pre_train_file = 'deepspeech-0.9.3-models.pbmm'
beam_width = 300
model = deepspeech.Model(model_file_path)

In [13]:
import wave
filename = 'example.wav'
w = wave.open(filename, 'r')
rate = w.getframerate()
frames = w.getnframes()
buffer = w.readframes(frames)
print(rate)
print(model.sampleRate())
type(buffer)

16000
16000


bytes

In [ ]:
# define function to read in sound file
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch
    
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

In [ ]:

# initialize constants
transcriptions = []
error = []
inference_time = []

len_ds = len(ds["text"])
BATCH_SIZE = 1



for i in range(len_ds//BATCH_SIZE):

    # import audio
    audio = ds["speech"][i*BATCH_SIZE:(i+1)*BATCH_SIZE]

    # count time of inference
    start = time.time()
    
    # tokenize
    input_values = processor(audio, sampling_rate=16000, return_tensors="pt", padding="longest").input_values  # Batch size 1

    # retrieve logits
    with torch.no_grad():
        logits = model(input_values).logits

    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)

    # store time of inference
    inference_time.append(time.time() - start)

    # store transcription
    transcriptions.append(transcription)

transcriptions = list(np.array(transcriptions).reshape(-1))

In [ ]:
df_transcriptions = pd.DataFrame(ds["text"][:len_ds], columns=['ground_truth'])
df_transcriptions[pre_train_file] = transcriptions
df_transcriptions[pre_train_file + "_inf_time"] = inference_time


df_transcriptions[pre_train_file] = df_transcriptions[pre_train_file].apply(lambda x: x.upper())

In [ ]:
df_transcriptions

In [19]:
import numpy as np
data16 = np.frombuffer(buffer, dtype=np.int16)
type(data16)
print(data16.shape)

(78480,)


In [15]:
text = model.stt(data16)
print(text)

he began ta confused complaint against the wizard who advantished behind the curtain on the left
